In [1]:
import os

import joblib
from keras.applications import ResNet50
from keras.applications.resnet import preprocess_input
import onnx
from keras.models import load_model, model_from_json, Model
import keras2onnx
from PIL import Image
import numpy as np
from glob import glob
from tqdm.notebook import tqdm

In [2]:
model = ResNet50()

In [13]:
image = Image.open('cat.jpg')
image = image.resize((224, 224))

In [14]:
img = np.array(image)

In [15]:
img = preprocess_input(img)

In [6]:
img.dtype

dtype('float32')

In [3]:
img = joblib.load('test_model/img.jbl')

In [4]:
cls = model.predict(img[None, :])

In [5]:
print(np.argmax(cls)), print(np.max(cls))

285
0.21655737


(None, None)

In [3]:
model = load_model(r"D:\Projects\英伟达备份\内镜精灵V7模型\已解密\imgs_nbi_ca\cache\weights2_1out.h5")

In [4]:
branch_model = Model(model.input, [model.layers[172].output, model.output])

In [5]:
branch_model.save(r"D:\Projects\英伟达备份\内镜精灵V7模型\已解密\imgs_nbi_ca\cache\weights2_old.h5")

In [3]:
onnx_model = keras2onnx.convert_keras(branch_model, branch_model.name, target_opset=11)
temp_model_file = r"D:\Projects\英伟达备份\内镜精灵V7模型\转换后\imgs_tool\cache\y_model1.onnx"
onnx.save_model(onnx_model, temp_model_file)

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 642 -> 178


# json加载模型

In [2]:
model = model_from_json(
    open(r"\\192.168.0.111\共享\for-wangwujin-1008\清晰判断2分类模型2020年\model1.json", 'r').read())
model.load_weights(r"\\192.168.0.111\共享\for-wangwujin-1008\清晰判断2分类模型2020年\weights1.hdf5")
model.save(r"\\192.168.0.111\共享\for-wangwujin-1008\清晰判断2分类模型2020年\weights1.h5")

# 批量转换h5到onnx

In [2]:
src_root = r'Y:\arm_linux\data_decrypt\data'

In [3]:
res50s = glob(os.path.join(src_root, '**','*.h5'), recursive=True)
res50s

['Y:\\arm_linux\\data_decrypt\\data\\imgs_crop\\cache\\y_model1.h5',
 'Y:\\arm_linux\\data_decrypt\\data\\imgs_in_out\\cache\\weights1.h5',
 'Y:\\arm_linux\\data_decrypt\\data\\imgs_nbi\\cache\\weights1.h5',
 'Y:\\arm_linux\\data_decrypt\\data\\imgs_parts\\cache\\weights3.h5',
 'Y:\\arm_linux\\data_decrypt\\data\\imgs_parts\\cache\\weights4.h5',
 'Y:\\arm_linux\\data_decrypt\\data\\imgs_xi\\cache\\weights2.h5',
 'Y:\\arm_linux\\data_decrypt\\data\\imgs_xi\\cache\\y_model1.h5']

In [3]:
res50s = [r"\\192.168.0.111\共享\for-wangwujin-1008\清晰判断2分类模型2020年\weights1.h5"]

In [4]:
for h5 in tqdm(res50s):
    model = load_model(h5)
    print(model.input_shape)
    onnx_model = keras2onnx.convert_keras(model, model.name, target_opset=11)
    onnx.save_model(onnx_model, h5[:-2] + 'onnx')

  0%|          | 0/1 [00:00<?, ?it/s]

tf executing eager_mode: True
tf.keras model eager_mode: False


(None, 224, 224, 3)


The ONNX operator number change on the optimization: 460 -> 128


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          640       
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 33,729
Trainable params: 33,729
Non-trainable params: 0
_________________________________________________________________


# 用trt将onnx转换成engine

In [1]:
import os
import pycuda.autoinit
import pycuda.driver as cuda
import tensorrt as trt
from tqdm.notebook import tqdm
from glob import glob

ModuleNotFoundError: No module named 'pycuda'

In [2]:
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
builder = trt.Builder(TRT_LOGGER)  # 创建一个builder
builder.max_batch_size = 1
config = builder.create_builder_config()  # 创建一个congig
config.max_workspace_size = 1 << 20
config.set_flag(trt.BuilderFlag.FP16)
# config.set_flag(trt.BuilderFlag.TF32)
EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

<ipython-input-2-8ffcc2737bd1>:3: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  builder.max_batch_size = 1
<ipython-input-2-8ffcc2737bd1>:5: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = 1 << 20


In [3]:
def onnx2trt(src_path, dst_path, input_shape=None):
    network = builder.create_network(EXPLICIT_BATCH)  # 创建一个network
    parser = trt.OnnxParser(network, TRT_LOGGER)
    # onnx_file_path = 'yolov3/yolo3-sim.onnx'
    model = open(src_path, 'rb')
    if not parser.parse(model.read()):
        for error in range(parser.num_errors):
            print(parser.get_error(error))
    # print(network.get_input(0).shape)
    # network.get_input(0).shape = [1,224,224,3]
    new_shape = list(network.get_input(0).shape)
    new_shape[0] = 1

    if input_shape is not None:
        network.get_input(0).shape = input_shape
    else:
        network.get_input(0).shape = new_shape
    print(network.get_input(0).shape)
    print('Completed parsing of ONNX file')
    print('Building an engine from file {}; this may take a while...'.format(src_path))
    plan = builder.build_serialized_network(network, config)
    with open(dst_path, "wb") as f:
        f.write(plan)
        print("Completed write Engine")

In [4]:
src_root = r'D:\Projects\bp-data-3.2.0'


In [9]:
onnxs = glob(os.path.join(src_root, '**', '*.onnx'), recursive=True)
onnxs

['\\\\192.168.0.60\\public\\产品二部\\胆胰大师模型TensorRT转化\\bp-data-3.2.0\\imgs_ercp_old\\cache\\weights256x256.onnx',
 '\\\\192.168.0.60\\public\\产品二部\\胆胰大师模型TensorRT转化\\bp-data-3.2.0\\imgs_ercp_old\\cache\\weights512x512.onnx']

In [5]:
onnxs = [r"D:\Downloads\WeChat Files\wxid_bw5xm0rdkitu22\FileStorage\File\2024-03\best.onnx"]

In [6]:
for ox in tqdm(onnxs):
    # if os.path.basename(ox) in ['y_model1.onnx','lstm_weights.onnx']:
    #     continue
    onnx2trt(ox, ox[:-4] + 'trt')

  0%|          | 0/1 [00:00<?, ?it/s]

(1, 3, 640, 640)
Completed parsing of ONNX file
Building an engine from file D:\Downloads\WeChat Files\wxid_bw5xm0rdkitu22\FileStorage\File\2024-03\best.onnx; this may take a while...
Completed write Engine


<ipython-input-4-3d2e8f988551>:2: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = 1 << 20


(1, 352, 352, 3)
(1, 224, 224, 3)


# 加载engine进行推理

In [6]:
with open(r"D:\Projects\znyx-plat\trunk\data_torch\20\imgs_wei_l_cls\weights1.trt", "rb") as f, trt.Runtime(
        TRT_LOGGER) as runtime:
    engine = runtime.deserialize_cuda_engine(f.read())

In [7]:
engine

In [19]:
engine_list = []
count1 = 0
while True:
    with open('test_model/res50.trt', "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
        temp = runtime.deserialize_cuda_engine(f.read())
        if temp is None:
            break
        engine_list.append(temp)
        count1 += 1


In [5]:
count1

69

In [20]:
for i in range(len(engine_list)):
    engine_list.pop(-1)
    # del a

In [18]:
del engine_list

In [10]:
engine_list

NameError: name 'engine_list' is not defined

In [6]:
import gc

In [7]:
gc.isenabled()

True

In [20]:
gc.collect()

4190

In [135]:
gc.get_count()

(428, 0, 1)

In [15]:
import sys

In [18]:
sys.getrefcount(engine_list[0])

NameError: name 'engine_list' is not defined

In [12]:
inputs = []
outputs = []
bindings = []
stream = cuda.Stream()
context = engine.create_execution_context()
for binding in engine:
    size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size

    dtype = trt.nptype(engine.get_binding_dtype(binding))

    # Allocate host and device buffers
    host_mem = cuda.pagelocked_empty(size, dtype)
    print(host_mem.dtype)
    device_mem = cuda.mem_alloc(host_mem.nbytes)

    # Append the device buffer to device bindings.
    bindings.append(int(device_mem))
    # Append to the appropriate list.
    if engine.binding_is_input(binding):
        inputs.append([host_mem, device_mem])
    else:
        outputs.append([host_mem, device_mem])


float32
float32


<ipython-input-12-967f2df3169a>:7: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size


In [12]:
# img = np.transpose(img,[2,0,1])

In [13]:
# import joblib


In [14]:
# joblib.dump(img,'zheng_model/img.jbl')

In [15]:
# import cv2
# import joblib
# img = cv2.imread('cat.jpg')
# img = cv2.resize(img,(224,224))
# img = img.astype(np.float)
# mean = [103.939, 116.779, 123.68]
# img[..., 0] -= mean[0]
# img[..., 1] -= mean[1]
# img[..., 2] -= mean[2]

In [8]:
import numpy as np
import cv2

In [14]:
def softmax(f):
    f -= np.max(f)
    return np.exp(f) / np.sum(np.exp(f))

In [9]:
# img = np.random.random((3, 224, 224))
img = np.random.random(( 224, 224,3))

In [10]:
img = img.astype(np.float32)

In [8]:
# joblib.dump(img,'test_model/img.jbl')
img = joblib.load('test_model/img.jbl')

In [9]:
img = cv2.imread(r"D:\Downloads\WeChat Files\wxid_bw5xm0rdkitu22\FileStorage\File\2023-06\f5472\f5472.jpg")
img = img[..., ::-1]
img = cv2.resize(img, (256, 256))
img = img[17:241, 17:241, :]
img = (img / 255 - [0.0482, 0.0437, 0.0387]) / [0.1193, 0.1182, 0.128]
img = np.transpose(img, (2, 0, 1))
img = img.astype(np.float32)
img = np.ascontiguousarray(img)

In [13]:
# trt_img = np.array(img[None,:],dtype=inputs[0][0].dtype,order='C')
inputs[0][0] = np.array(img[None, :], dtype=inputs[0][0].dtype, order='C')
# trt_img = img[None,:].ravel()

In [14]:
[cuda.memcpy_htod_async(inp[1], inp[0], stream) for inp in inputs]
# Run inference.
context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
# Transfer predictions back from the GPU.
[cuda.memcpy_dtoh_async(out[0], out[1], stream) for out in outputs]
# Synchronize the stream
stream.synchronize()

In [15]:
[(np.argmax(out[0]), np.max(out[0])) for out in outputs]

[(2, 0.50871706)]

In [17]:
outputs[0][0]

array([0.38740793, 0.07354643, 0.50871706, 0.01728551, 0.01304315],
      dtype=float32)

In [19]:
outputs[0][1]

In [15]:
softmax(outputs[0][0])

array([9.9495232e-01, 1.1161991e-03, 3.5055049e-03, 1.0085311e-04,
       1.9215094e-04, 4.7181980e-05, 5.4983484e-05, 3.0327721e-05,
       4.6079029e-07], dtype=float32)

In [13]:
import time

In [23]:
start = time.time()
img_in = np.array(img[None, :], dtype=inputs[0][0].dtype, order='C')
for i in range(10000):
    inputs[0][0] = img_in
    [cuda.memcpy_htod_async(inp[1], inp[0], stream) for inp in inputs]
    # Run inference.
    context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out[0], out[1], stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
end = time.time()

In [24]:
(end - start) / 10000

0.002201115107536316